##  Random Number Generation

In [1]:
import numpy as np

In [2]:
def sn_random_numbers(shape, antithetic=True, moment_matching=True,
                     fixed_seed=False):
    
    """Returns an array of shape shape with (pseudo)random numbers
    that are statndard normally distributed.
    
    Parameters
    =========
    shape : tuple (o, n, m)
        generation of array with shape (o, n, m)
        
        
    Results
    =========
    ran : (o,)
    """
    
    if fixed_seed:
        np.random.seed(1000)
    if antithetic:
        ran = np.random.standard_normal((shape[0], shape[1], shape[2] / 2))
        ran = np.concatenate((ran, -ran), axis = 2)
    else:
        ran = np.random.standard_normal(shape)
    if moment_matching:
        ran = ran - np.mean(ran)
        ran = ran / np.std(ran)
    if shape[0] == 1:
        return ran[0]
    else:
        return ran
    

In [3]:
snrn = sn_random_numbers((2,2,2), antithetic=False,
                        moment_matching=False,
                        fixed_seed=True)

In [4]:
snrn

array([[[-0.8044583 ,  0.32093155],
        [-0.02548288,  0.64432383]],

       [[-0.30079667,  0.38947455],
        [-0.1074373 , -0.47998308]]])

In [5]:
snrn_mm = sn_random_numbers((2, 3, 2), antithetic=False,
                           moment_matching=True,
                           fixed_seed=True)

In [6]:
snrn_mm

array([[[-1.47414161,  0.67072537],
        [ 0.01049828,  1.28707482],
        [-0.51421897,  0.80136066]],

       [[-0.14569767, -0.85572818],
        [ 1.19313679, -0.82653845],
        [ 1.3308292 , -1.47730025]]])

In [7]:
snrn_mm.mean()

3.7007434154171883e-17

In [8]:
snrn_mm.std()

1.0

##  Generic Simulation Class

In [9]:
import numpy as np
import pandas as pd

In [13]:
class simulation_class(object):
    """ Providing base methods for simulation classes.
    
    Attributes
    ==========
    name : string
    mar_env : instance of market_environment
    
    corr : Boolean
    
    
    
    Methods
    =======
    generate_time_grid : 
    
    get_instrument_values :
    """
    
    
    
    def __init__(self, name, mar_env, corr):
        try:
            self.name = name
            self.pricing_date = mar_env.pricing_date
            self.initial_value = mar_env.get_constant('initial_value')
            self.volatility = mar_env.get_constant('volatility')
            self.final_date = mar_env.get_constant('final_date')
            self.currency = mar_env.get_constant('currency')
            self.frequency = mar_env.get_constant('frequency')
            self.paths = mar_env.get_constant('paths')
            self.discount_curve = mar_env.get_curve('discount_curve')
            
            try:
                # if time_grid in mar_env take this
                # (for portfolio valuetion
                self.time_grid = mar_env.get_list('time_grid')
            except:
                self.time_grid = None
                
            try:
                # if there are special dates, then add these
                self.special_dates = mar_env.get_list('special_dates')
            except:
                self.special_dates = []
                
            self.instrument_values = None
            self.correlated = corr
            
            if corr is True:
                # only needed in a portfolio context when
                # risk factors are correlated
                self.cholesky_matrix = mar_env.get_list('cholesky_matrix')
                self.rn_set = mar_env.get_list('rn_set')[self.name]
                self.random_numbers = mar_env.get_list('random_numbers')
        except:
            print("Error parsing market environment.")
                
        
    def generate_time_grid(self):
        start = self.pricing_date
        end = self.final_date
        # pandas date_range function
        
        time_grid = pd.date_range(start=start, end=end,
                                  freq=self.frequency).to_pydatetime()
        time_grid = list(time_grid)
        # enhance time_grid by start, end, and special_dates
        if start not in time_grid:
            time_grid.insert(0, start)
            
        if end not in time_grid:
            time_grid.append(end)
            
        if len(self.special_dates) > 0:
            
            time_grid.extend(self.special_dates)
            
            time_grid = list(set(time_grid))
            
            time_grid.ort()
        
        self.time_grid = np.array(time_grid)
            
        def get_instrument_values(self, fixed_seed=True):
            if self.instrument_values is None:
                
                self.generate_paths(fixed_seed=fixed_seed, day_count=365.)
                
            elif fixed_seed is False:
                
                self.generate_paths(fixed_seed=fixed_seed, day_count=365.)
            
            return self.instrument_values
        
                         
        
        
        
        
    
    

## Geometric Brownian Motion

### The Simulation Class

In [14]:
import numpy as np

In [ ]:
from sn_random_numbers import sn_random_numbers
from simulation_class import simulation_class